# Video Style Transfer

### Install dependencies
- Install [VLC](https://www.videolan.org/)
- `brew install ffmpeg`
- Install [OpenVINO Development](https://docs.openvino.ai/latest/openvino_docs_install_guides_install_dev_tools.html) incl. ONNX API

### Activate virtual environment
- `source ~/openvino_env/bin/activate`

### Import packages

In [1]:
import sys
import platform
from enum import Enum
from pathlib import Path
from glob import glob

import cv2
import numpy as np
from openvino.runtime import Core, PartialShape
from yaspin import yaspin

sys.path.append("/Users/tompoek/openvino_env/notebooks/utils")
from notebook_utils import download_file

### Download pretrained models

In [2]:
BASE_URL = "https://github.com/onnx/models/raw/main/vision/style_transfer/fast_neural_style/model"
MODEL_DIR = "model"

class Style(Enum):
    MOSAIC = "mosaic"
    RAIN_PRINCESS = "rain-princess"
    CANDY = "candy"
    UDNIE = "udnie"
    POINTILISM = "pointilism"

    def __init__(self, *args):
        self.model_path = Path(f"{self.value}-9.onnx")
        self.title = self.value.replace("-", " ").title()
        self.url = f"{BASE_URL}/{self.model_path}"

In [3]:
for style in Style:
    if not Path(f"{MODEL_DIR}/{style.model_path}").exists():
        download_file(style.url, directory=MODEL_DIR)

### Set video file and frame rate

In [4]:
VIDEO_FILE = "MVI_2540.MP4"
video_name = Path(VIDEO_FILE).stem
fps_video, fps_frames = 25., 25.  # Hz
fps_ratio = fps_video / fps_frames

### Extract frame images from video

In [5]:
# Detect VLC command per OS
if platform.system() == "Windows":
    vlc = "\"C:\\Program Files\\VideoLAN\\VLC\\vlc.exe\""
elif platform.system() == "Darwin":  # OSX
    vlc = "/Applications/VLC.app/Contents/MacOS/VLC"
else:  # Linux
    vlc = "vlc"

# Prepare images folder
if Path(video_name).exists():
    remove_dir_command = f"rm -rf {video_name}"
    ! $remove_dir_command
make_dir_command = f"mkdir {video_name}"
! $make_dir_command

# Extract frame images
extract_frames_command = f"{vlc} {VIDEO_FILE} --video-filter=scene " \
                         f"--scene-ratio={fps_ratio} --rate={fps_ratio} " \
                         f"--vout=dummy --aout=dummy --intf=dummy " \
                         f"--scene-path={video_name} " \
                         f"--scene-format=jpeg " \
                         f"--quiet vlc://quit"
! $extract_frames_command

VLC media player 3.0.17.3 Vetinari (revision 3.0.17.3-0-g426513d88e)
[swscaler @ 0x7f9d230cb000] deprecated pixel format used, make sure you did set range correctly


### Transfer images to artistic styles

In [6]:
# (Optional) Resize images to smaller size
def resize_to_max(image: np.ndarray, max_side: int) -> np.ndarray:
    """
    Resize image to an image where the largest side has a maximum length of max_side
    while keeping aspect ratio. Example: if an original image has width and height of (1000, 500)
    and max_side is 300, the resized image will have a width and height of (300, 150).

    :param image: Array of image to resize
    :param max_side: Maximum length of largest image side
    :return: Resized image
    """
    if max(image.shape) <= max_side:
        new_image = image
    else:
        index = np.argmax(image.shape)
        factor = max_side / image.shape[index]
        height, width = image.shape[:2]
        new_height, new_width = int(factor * height), int(factor * width)
        new_image = cv2.resize(image, (new_width, new_height))
    return new_image

In [7]:
IMAGE_FILES = glob(f"{video_name}/scene*.jpeg")

# (Optional) Resize images for efficiency
RESIZE_IMAGE = False
resized_max_side = 800

# Create a `Core` instance.
ie = Core()

# Loop over models to transfer images' styles.
for i, style in enumerate(Style):
    # Prepare output folder
    output_folder = Path(f"{video_name}_{style.model_path.stem}")
    output_folder.mkdir(exist_ok=True)
    
    # Load the model and get model info.
    model = ie.read_model(model=Path(MODEL_DIR) / style.model_path)
    input_key = model.input(0)
    
    # Load an arbitrary image to extract its shape for network prep
    image = cv2.cvtColor(cv2.imread(IMAGE_FILES[0]), cv2.COLOR_BGR2RGB)
    if RESIZE_IMAGE:
        image = resize_to_max(image=image, max_side=resized_max_side)

    # Reshape the network to the image shape and load the network to a device.
    model.reshape({input_key: PartialShape([1, 3, image.shape[0], image.shape[1]])})
    compiled_model = ie.compile_model(model=model, device_name="CPU")
    output_key = compiled_model.output(0)

    with yaspin(text=f"Transferring images to {style.title} style") as sp:
        # Loop over images to transfer style.
        for image_file in IMAGE_FILES:
            image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
            if RESIZE_IMAGE:
                image = resize_to_max(image=image, max_side=resized_max_side)
            
            # Transpose the input image to network dimensions
            input_image = np.expand_dims(image.transpose(2, 0, 1), axis=0)
            # Extract the name and the shape of the image.
            image_name = Path(image_file).stem
            image_shape_str = f"{image.shape[1]}x{image.shape[0]}"
            result = compiled_model([input_image])[output_key]

            # Convert the inference result to the image shape and apply postprocessing.
            # Postprocessing is described in the model documentation:
            # https://github.com/onnx/models/tree/master/vision/style_transfer/fast_neural_style
            result = result.squeeze().transpose(1, 2, 0)
            result = np.clip(result, 0, 255).astype(np.uint8)

            # Save result to image
            image_path = f"{image_name}.jpeg"
            output_path = output_folder / image_path
            cv2.imwrite(str(output_path), cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        
        # Display check mark
        sp.ok("✔")

    del model
    del compiled_model


✔ Transferring images to Mosaic style25h
✔ Transferring images to Rain Princess style25h
✔ Transferring images to Candy style
✔ Transferring images to Udnie style
✔ Transferring images to Pointilism style


### Reconstruct video from styled images

In [8]:
# Loop over models to reconstruct video from styled images.
for i, style in enumerate(Style):
    # Read output folder
    output_folder = Path(f"{video_name}_{style.model_path.stem}")
    
    with yaspin(text=f"Creating video in {style.title} style") as sp:
        # Reconstruct video from images
        reconstruct_video_command = f"ffmpeg -r {fps_frames} " \
                                    f"-i {output_folder}/scene%05d.jpeg " \
                                    f"{output_folder}.mp4 " \
                                    f"-v quiet"
        ! $reconstruct_video_command
        
        # Display check mark
        sp.ok("✔")

✔ Creating video in Mosaic style25h
✔ Creating video in Rain Princess style
✔ Creating video in Candy style
✔ Creating video in Udnie style
✔ Creating video in Pointilism style
